In [1]:
import pickle
import nltk
import random
import re
from scipy.spatial.distance import cosine
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shubham/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def get_pseudo_sentence(sent,noun_phrase_list,dist_matrix):
    tagged_sent = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(sent))[0]
    sent = ''
    for tup in tagged_sent:
        #print(tup)
        if tup[1] in ['NN','NNS'] and len(tup[0]) > 3:
            #find top-10 most similar for this key-phrase
            np_index = noun_phrase_list.index(tup[0])
            #top-10 similar indices
            top_10_idx = list((-dist_matrix[np_index]).argsort()[:15])
            random_idx = random.randint(1,9)
            rand_word = noun_phrase_list[top_10_idx[random_idx]]
            sent = sent+' '+rand_word
        else:
            sent = sent+' '+tup[0]
    return sent.strip().replace(' .','.').replace(' ,',',').replace(' ?','?')

In [3]:
random_sentences = pickle.load(open('../../datasets/random_sents.pkl','rb'))
random_sentences = [ele for ele in random_sentences if len(ele.split(' '))>2]

action_items_file = open("../../datasets/sample_actions.txt", "r")
action_items_list = action_items_file.readlines()
action_items_list = [ele.strip() for ele in action_items_list]
dft = pd.read_csv('/home/shubham/hdd/shubham/projects/action_item_detection/Shubham/training/seed_data/seed_train.csv')
action_items_list = list(dft[dft['is_action_item']==1]['text'])
random_sentences = list(dft[dft['is_action_item']==0]['text'])
merged_text = action_items_list+random_sentences

In [4]:
#print(random_sentences)

In [5]:
noun_phrase_list = []
md_list = []
for text in merged_text:
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))[0]
    noun_phrase_list+=[ele[0] for ele in tagged_sents if ele[1] in ['NN','NNS']]
    md_list+=[ele[0] for ele in tagged_sents if ele[1] in ['MD']]
    #print(set(md_list))
    
noun_phrase_list = list(set(noun_phrase_list))
noun_phrase_list = [ele for ele in noun_phrase_list if len(ele)>3]


In [6]:
embed = hub.Module("/home/shubham/hdd/Venkat/knowledge_graphs/tfhub_models/")
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    noun_phrase_embeddings = session.run(embed(noun_phrase_list))
    
master_list = []
for ele1 in noun_phrase_embeddings:
    curr_list = []
    for ele2 in noun_phrase_embeddings:
        curr_list.append(1-cosine(ele1,ele2))
    master_list.append(curr_list)
    
dist_matrix = np.array(master_list)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
#generate random action item sentences
pseudo_ai_list = []
actual_list = []
for i in range(100000):
    curr_sent = action_items_list[random.randint(0,len(action_items_list)-1)]
    #print(curr_sent)
    actual_list.append(curr_sent)
    #print(get_pseudo_sentence(curr_sent,noun_phrase_list,dist_matrix))
    #print('#########################')
    pseudo_ai_list.append(get_pseudo_sentence(curr_sent,noun_phrase_list,dist_matrix))

In [8]:
pseudo_non_ai_list = []
actual_list = []
for i in range(100000):
    curr_sent = random_sentences[random.randint(0,len(random_sentences)-1)]
    actual_list.append(curr_sent)
    pseudo_non_ai_list.append(get_pseudo_sentence(curr_sent,noun_phrase_list,dist_matrix))

In [9]:
ai_list = list(set(pseudo_ai_list+action_items_list))
non_ai_list = list(set(pseudo_non_ai_list+random_sentences))
merged_text = ai_list+non_ai_list
label_list = [1]*len(ai_list)+[0]*len(non_ai_list)

In [10]:
train_df = pd.DataFrame({'text': merged_text,
                        'is_action_item': label_list})
train_df = train_df.sample(frac=1)
test_df = train_df[0:1000]
train_df = train_df[1000:]
#shuffle labels for sanity check
#lab_list = list(train_df['is_action_item'])
#random.shuffle(lab_list)
#train_df['is_action_item'] = lab_list

In [11]:
train_df.to_csv('new_train.csv', index=False)
test_df.to_csv('new_test.csv', index=False)

In [12]:
merged_list = action_items_list+random_sentences[0:1000]
labs = [1]*len(action_items_list)+[0]*len(random_sentences[0:1000])
actual_train_df = pd.DataFrame({'text': merged_list,
                        'is_action_item': labs})
#actual_train_df.to_csv('train_act.csv', index=False)

In [13]:
ai_list = ["Let us use lambda for deploying any AI services moving forward",
           "When you are there, make sure you report the performance metrics",
           "Can one of you take the database backup over the weekend",
           "Let us plan for the weekend support after our first customer",
           "Send the space probe to mars by 2020","We have to put the plan for campaign management",
           "Please speedup the hiring process","Make sure, you are following up with the candidate",
           "We should proceed with deploying the service as a container",
           "Let us focus on the P1 issue for next 4 hours","Drop the kid at school","Eat the frog",
           "We should put the model version control process in place before we scale up the team",
           "Let’s look into it once we are done with the call","What is the plan for jitsi migration",
           "Work from home tomorrow"]

nonai_list = ["Machine Learning is the way we are making things better","I have completed the UI pipeline.",
              "It can help us in managing the models better",
              "Campaign management has gone well. We have traction with lot of customers",
              "Marketing teams are always looking for better visualisation tools",
              "How did the weekend go?",
              "There was a huge traffic in the city center, I got stuck there for 15 mins",
              "International space station can be seen from Bangalore today",
              "I am flying to Mumbai today","Cullen is visiting us in the first week of January",
              "He is very good at UI ad UX ","Action item detection is an important feature of Ether",
             "You are awesome man."]

merged_list = ai_list+nonai_list
labels = [1]*len(ai_list)+[0]*len(nonai_list)

df_test = pd.DataFrame({'text': merged_list,
                       'isAI': labels})
#df_test.to_csv('random_test.csv', index=False)

In [14]:
text_list1 = ['This will create a big issue unless we act, customers are going to complain so please fix that bug by tomorrow or we might have to anounce this around',
             'campaign management is crutial and we might not be able to take off without a robust campaign, please make it successful so that we can work on traction that we generate out of it',
            'Our hiring process looks very week, we have to optimize the hiring process so that we do not create any bottleneck in the process',
             'What I have observed is that it is happening during Jitsi call, please debug the issue during the live call so that we can find the table that is creating this issue',
            'To see this actually working, create a marker and match them with auto generated ones so that the user will be able to experience the true strength of Ether features']

text_list2 = ["Calling developers and researchers to hack together packages, demos and applications on top of PyTorch and submit online, we’ll help you reach the community, amplify your vision and have $60K in prizes!",
             "For accomplishing such a task, you need both sentence tokenization as well as words to calculate the ratio, such output serves as an important feature for machine training as the answer would be numeric.",
             "We have taken the same sentence. Further sent module parsed that sentences and show output, it is clear that this function breaks each sentence.",
             "Unfortunately, for many starting out in NLP and even for some experienced practicioners, the theory and practical application of these powerful models is still not well understood.",
             "To better explain AllenNLP and the concepts underlying the framework, I will first go through an actual example using AllenNLP to train a simple text classifier."]

merged_list = text_list1+text_list2
labels = [1]*len(text_list1)+[0]*len(text_list2)

df_test2 = pd.DataFrame({'text': merged_list,
                       'isAI': labels})
df_test2.to_csv('random_long_test.csv', index=False)

In [15]:
all_test = pd.concat([df_test,df_test2])
#all_test.to_csv('mixed_tests.csv', index=False)

In [16]:
acti = actual_train_df[actual_train_df['is_action_item']==1]

acti = list(acti['text'])
print(acti[0])

take away the sponsors from Pandya


In [17]:
#import counter from collections
from collections import Counter

for i in acti:
    #print(nltk.word_tokenize(i)[0])
    print(nltk.pos_tag(nltk.word_tokenize(i)))
    

[('take', 'VB'), ('away', 'RP'), ('the', 'DT'), ('sponsors', 'NNS'), ('from', 'IN'), ('Pandya', 'NNP')]
[('Please', 'VB'), ('connect', 'NN'), ('to', 'TO'), ('Zoom', 'NNP'), ('account', 'NN'), ('representative', 'NN'), ('to', 'TO'), ('get', 'VB'), ('details', 'NNS')]
[('move', 'NN'), ('to', 'TO'), ('the', 'DT'), ('next', 'JJ'), ('point', 'NN')]
[('look', 'NN'), ('into', 'IN'), ('knowledge', 'NN'), ('graphs', 'NN')]
[('Share', 'NNP'), ('exact', 'JJ'), ('location', 'NN'), ('for', 'IN'), ('the', 'DT'), ('conference', 'NN'), ('.', '.')]
[('waiting', 'VBG'), ('for', 'IN'), ('some', 'DT'), ('results', 'NNS'), ('to', 'TO'), ('be', 'VB'), ('shown', 'VBN'), ('to', 'TO'), ('me', 'PRP'), ('on', 'IN'), ('this', 'DT')]
[('Let', 'VB'), ("'s", 'POS'), ('conquer', 'VB'), ('the', 'DT'), ('world', 'NN')]
[('Make', 'VB'), ('the', 'DT'), ('change', 'NN'), ('to', 'TO'), ('the', 'DT'), ('settings', 'NNS'), ('menu', 'VBP'), ('per', 'IN'), ('the', 'DT'), ('latest', 'JJS'), ('design', 'NN'), ('.', '.'), ('We', 

[('you', 'PRP'), ('need', 'VBP'), ('close', 'RB'), ('this', 'DT'), ('off', 'RP')]
[('try', 'VB'), ('working', 'VBG'), ('with', 'IN'), ('shubham', 'NN'), ('and', 'CC'), ('give', 'VB'), ('him', 'PRP'), ('heads', 'VB'), ('up', 'RP')]
[('Please', 'NNP'), ('create', 'VB'), ('a', 'DT'), ('design', 'NN'), ('document', 'NN'), ('for', 'IN'), ('this', 'DT'), ('new', 'JJ'), ('feature', 'NN'), ('and', 'CC'), ('share', 'NN'), ('with', 'IN'), ('the', 'DT'), ('team', 'NN')]
[('Can', 'MD'), ('we', 'PRP'), ('change', 'VB'), ('STUN', 'NNP'), ('to', 'TO'), ('Co-Turn', 'NNP'), ('in', 'IN'), ('production', 'NN'), ('?', '.')]
[('Create', 'VB'), ('a', 'DT'), ('new', 'JJ'), ('dashboard', 'NN'), ('in', 'IN'), ('scalyr', 'NN')]
[('IOS', 'NNP'), ('interstitial', 'JJ'), ('routing', 'VBG'), ('-', ':'), ('Karthik', 'NNP'), ('will', 'MD'), ('write', 'VB'), ('a', 'DT'), ('1-pager', 'JJ'), ('and', 'CC'), ('Tushar', 'NNP'), ('will', 'MD'), ('implement', 'VB')]
[('last', 'JJ'), ('participant', 'JJ'), ('logic', 'NN'), ('

[('implement', 'JJ'), ('retry', 'NN'), ('mechanism', 'NN'), ('for', 'IN'), ('nats', 'NNS'), ('client', 'NN')]
[('Evaluate', 'NNP'), ('and', 'CC'), ('recommend', 'NN'), ('tools', 'NNS'), (',', ','), ('technologies', 'NNS'), ('and', 'CC'), ('processes', 'NNS'), ('to', 'TO'), ('ensure', 'VB'), ('the', 'DT'), ('highest', 'JJS'), ('quality', 'NN'), ('product', 'NN'), ('platform', 'NN')]
[('Brother', 'CC'), ('one', 'CD'), ('more', 'JJR'), ('thing', 'NN'), ('there', 'RB'), ('was', 'VBD'), ('a', 'DT'), ('thread', 'NN'), ('that', 'WDT'), ('started', 'VBD'), ('about', 'IN'), ('putting', 'VBG'), ('heartfulness', 'NN'), ('videos', 'NNS'), ('and', 'CC'), ('content', 'NN'), ('on', 'IN'), ('hola', 'NN'), ('cabs', 'NN'), ('.', '.'), ('Yeah', 'UH'), (',', ','), ('and', 'CC'), ('we', 'PRP'), ('actually', 'RB'), ('started', 'VBD'), ('this', 'DT'), ('process', 'NN'), ('long', 'JJ'), ('time', 'NN'), ('ago', 'IN'), ('when', 'WRB'), ('I', 'PRP'), ('was', 'VBD'), ('in', 'IN'), ('Nola', 'NNP'), (',', ','), ('b

[('Ensure', 'NNP'), ('data', 'NNS'), ('center', 'NN'), ('related', 'VBN'), ('priorities', 'NNS'), ('are', 'VBP'), ('consistent', 'JJ'), ('with', 'IN'), ('company', 'NN'), ('objectives', 'NNS'), ('.', '.')]
[('address', 'NN'), ('these', 'DT'), ('first', 'JJ'), (',', ','), ('then', 'RB'), ('move', 'VB'), ('on', 'IN'), ('to', 'TO'), ('the', 'DT'), ('next', 'JJ'), ('task', 'NN')]
[('Look', 'VB'), ('into', 'IN'), ('Janus', 'NNP'), ('commercial', 'JJ'), ('license', 'NN')]
[('Identify', 'NNP'), ('regional', 'JJ'), ('trends', 'NNS'), ('and', 'CC'), ('issues', 'NNS'), ('on', 'IN'), ('social', 'JJ'), ('that', 'WDT'), ('have', 'VBP'), ('global', 'JJ'), ('potential', 'NN'), ('and', 'CC'), ('flag', 'NN'), ('to', 'TO'), ('coordinator', 'VB')]
[('Can', 'MD'), ('you', 'PRP'), ('please', 'VB'), ('help', 'VB'), ('Jidesh', 'NNP'), ('get', 'VB'), ('the', 'DT'), ('merchant', 'NN'), ('dump', 'NN'), ('from', 'IN'), ('Shoptiq', 'NNP'), ('to', 'TO'), ('test', 'VB'), ('wirh', 'NN'), ('Sharetribe', 'NNP'), ('?',

[('Next', 'JJ'), ('Steps', 'NNS'), (':', ':'), ('1', 'CD'), (')', ')'), ('Create', 'NNP'), ('Developer', 'NNP'), ('Account', 'NNP'), (',', ','), ('2', 'CD'), (')', ')'), ('Partner', 'NNP'), ('Agreement', 'NNP'), ('via', 'IN'), ('Mark', 'NNP')]
[('I', 'PRP'), ('will', 'MD'), ('post', 'VB'), ('the', 'DT'), ('results', 'NNS'), ('by', 'IN'), ('the', 'DT'), ('end', 'NN'), ('of', 'IN'), ('the', 'DT'), ('day', 'NN')]
[('book', 'NN'), ('the', 'DT'), ('meeting', 'NN'), ('room', 'NN'), ('for', 'IN'), ('next', 'JJ'), ('two', 'CD'), ('days', 'NNS'), ('and', 'CC'), ('you', 'PRP'), ('can', 'MD'), ('walk', 'VB'), ('us', 'PRP'), ('through', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('approach', 'NN'), ('for', 'IN'), ('handling', 'VBG'), ('failed', 'JJ'), ('cases', 'NNS')]
[('change', 'NN'), ('the', 'DT'), ('horizontal', 'JJ'), ('scroll', 'NN'), ('view', 'NN'), ('to', 'TO'), ('vertical', 'VB')]
[('Oh', 'UH'), ('i', 'NN'), ('forgot', 'VBD'), ('to', 'TO'), ('close', 'VB'), ('this', 'DT'), ('issue', 'NN'), (',

In [18]:
train_df.groupby('is_action_item').agg('count')
#actual_train_df

,text
is_action_item,
0,67117
1,58973
